In [1]:
import pandas as pd
df = pd.read_csv('../../top_rated_wines.csv')
df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
data = df.sample(700).to_dict('records') # Get only 700 records. More records will make it slower to index
len(data)

700

In [2]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [3]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [4]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [5]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

/tmp/ipykernel_90173/3075695827.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [6]:
# vectorize!
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [11]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

In [12]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'Catena Zapata Argentino Vineyard Malbec 2004', 'region': 'Argentina', 'variety': 'Red Wine', 'rating': 98.0, 'notes': '"The single-vineyard 2004 Malbec Argentino Vineyard spent 17 months in new French oak. Remarkably fragrant and complex aromatically, it offers up aromas of wood smoke, creosote, pepper, clove, black cherry, and blackberry. Made in a similar, elegant style, it is the most structured of the three single vineyard wines, needing a minimum of a decade of additional cellaring. It should easily prove to be a 25-40 year wine. It is an exceptional achievement in Malbec. When all is said and done, Catena Zapata is the Argentina winery of reference – the standard of excellence for comparing all others. The brilliant, forward-thinking Nicolas Catena remains in charge, with his daughter, Laura, playing an increasingly large role. The Catena Zapata winery is an essential destination for fans of both architecture and wine in Mendoza. It is hard to believe, given the surge i

In [10]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [17]:
# Now time to connect to the local large language model
#from openai import OpenAI
#client = OpenAI(
#    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
#    api_key = "sk-no-key-required"
#)
#completion = client.chat.completions.create(
#    model="LLaMA_CPP",
#    messages=[
#        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
#        {"role": "user", "content": "Suggest me an amazing Malbec wine from Argentina"},
#        {"role": "assistant", "content": str(search_results)}
#    ]
#)
#print(completion.choices[0].message)

In [16]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required" # An API key is not required!
)
completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests. Please answer with sentences."},
        {"role": "user", "content": "Suggest me an amazing Malbec wine from Argentina"},
        {"role": "assistant", "content": f"Based on the search results: {str(search_results)}, please suggest an amazing Malbec wine from Argentina and provide a brief description."}
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='Based on the search results, I would suggest the "Greenock Creek Alices Shiraz 2001" from the Barossa Valley, Barossa, South Australia, Australia. This wine is a Malbec and it has a rating of 98.0, which indicates that it is an amazing wine. The wine has rich and fleshy flavors with notes of coffee, plum, wild berry and spice, and it is smooth and polished with a long and flavorful finish. It is an extremely limited release wine from one of Australia\'s "Cult" wineries, which means it is a highly sought-after and sought-after wine.</s>', role='assistant', function_call=None, tool_calls=None)
